#### Import需要的library

In [1]:
import numpy as np
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from scipy.special import expit
import cv2
import os
from os import listdir
from os.path import isfile, join

#### 讀取dataset並回傳圖片跟label的函式
* label_folders: 要讀取的labels
* dataset_path: 要讀取的資料集路徑
* preprocessing: 是否要做一些圖片的前處理，主要是把圖片轉成1D vector並normalize到0~1之間
* encoding: 是否要對labels做one hot encoding

In [2]:
def dataset(label_folders, dataset_path='MNIST/train', preprocessing=True, encoding=True, rotate=False):
  img_list = []
  label_list = []
  enc = OneHotEncoder(handle_unknown='ignore')
  label_folders = np.array(label_folders).reshape(-1, 1)
  enc.fit(label_folders)
  
  # print(enc.categories_)

  for l in label_folders:
    labelPath = join(dataset_path, str(l.item()))
    file_names = [f for f in listdir(labelPath) if isfile(join(labelPath, f))]

    for file_name in file_names:
      img_path = join(labelPath, file_name)
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

      if preprocessing:
        if rotate:
          img = cv2.rotate(img, cv2.ROTATE_180) # 不知為何助教給的圖片轉了180度，所以在這裡轉回來
        img = img.reshape(-1).astype('float32')
        img /= 255.0
        
      l_enc = l  
      if encoding:
        # print(l)
        l_enc = enc.transform([l]).toarray()[0]
        # print(l_enc)
      
      img_list.append(img)
      label_list.append(l_enc)

  return np.array(img_list), np.array(label_list)

def test_dataset(dataset_path, preprocessing=True):
  def takeSecond(elem):
    return elem[1]
  
  data_list = []
  
  file_names = [f for f in listdir(dataset_path)]
  file_names.sort()
  
  for file_name in file_names:
    img_path = join(dataset_path, file_name) # 'mnist_testData/Tesing_data/0000.png'
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    if preprocessing:
      img = cv2.rotate(img, cv2.ROTATE_180)
      img = img.reshape(-1).astype('float32')
      img /= 255.0
    
    data_list.append((img, file_name))
    # data_list.sort(key=takeSecond)
    
  return data_list

#### 讀取dataset，讓它回傳數字2, 4, 5, 6, 7的圖片和labels
* images shape: [number of images, height * width]
* labels shape: [number of labels, number of classes]

In [3]:
from sklearn.model_selection import train_test_split
classes = [2, 4, 5, 6, 7]
training_images, training_labels = dataset(
    classes, dataset_path='hw1/Training_data', rotate=True)

valid_images, valid_labels = dataset(classes,
                                     dataset_path='hw1/MNIST/valid',
                                     encoding=False)


In [4]:
print(training_images.shape, training_labels.shape, valid_images.shape,
      valid_labels.shape)


(20000, 784) (20000, 5) (4892, 784) (4892, 1)


#### $Sigmoid(x) = \frac{1}{1 + \exp^{-x}}$

In [5]:
def sigmoid(x):
  # return 1.0 / (1.0 + np.exp(-x))
  return expit(x) # avoid overflow

#### 搭建model為一個class
* feed_forward: 做feed forward
* train_model: 丟入dataset和超參數來訓練模型
* update: 利用back propagation更新模型
* back_propagation: 計算$\Delta w$和$\Delta b$
* predict: 預測圖片的label值
* compute_cost: 計算error，也就是講義裡的$\delta$

In [6]:
from tqdm.auto import tqdm

class neural_network():
    def __init__(self, sizes):
        """suppose sizes=[2, 3, 4], weights would be [3, 2], [4, 3], biases would be [3, 1], [4, 1]"""
        self.sizes = sizes
        self.num_layer = len(self.sizes)
        self.weights = [np.random.randn(self.sizes[i], self.sizes[i-1]) for i in range(1, self.num_layer)]
        self.biases = [np.random.randn(self.sizes[i], 1) for i in range(1, self.num_layer)]
        self.last_delta_w = [np.zeros((self.sizes[i], self.sizes[i-1])) for i in range(1, self.num_layer)]
        self.last_delta_b = [np.zeros((self.sizes[i], 1)) for i in range(1, self.num_layer)]
        self.momentum = 0.0

    def feed_forward(self, a):
        for w, b in zip(self.weights, self.biases):
            a = sigmoid(np.dot(w, a) + b)

        return a

    def train_model(self, images, labels, num_epoch, learning_rate, batch_size, valid_images=None, valid_labels=None, momentum=0.0):
        f = open('project1_acc.txt', 'w')

        n = len(images)
        self.momentum = momentum

        for epoch in tqdm(range(num_epoch)):

            images, labels = shuffle(images, labels)

            mini_batches_images = [images[i:i+batch_size] for i in range(0, n, batch_size)]
            mini_batches_labels = [labels[i:i+batch_size] for i in range(0, n, batch_size)]

            for image, label in zip(mini_batches_images, mini_batches_labels):
                self.update(learning_rate, image,label)

            # Calculate test data accuracy
            if valid_images is not None:
                acc = 0.0
                for image, label in zip(valid_images, valid_labels):
                    result = self.predict(image)
                    if classes[result] == label.item():
                        acc += 1
                print('Epoch {} / {}     Accuracy : {}'.format(epoch+1, num_epoch, acc/len(valid_images)))

                f.write('{}\n'.format((acc / len(valid_images))*100))

        f.close()


    def update(self, learning_rate, image, label):

        delta_weight, delta_bias = self.back_propagation(image, label)

        self.weights = [w + learning_rate * delta_w/len(image) for w, delta_w in zip(self.weights, delta_weight)]
        self.biases = [b + learning_rate * delta_b/len(image) for b, delta_b in zip(self.biases, delta_bias)]

        self.last_delta_w = [w for w in delta_weight]
        self.last_delta_b = [b for b in delta_bias]


    def back_propagation(self, image, label):
        delta_weights = [np.zeros(w.shape) for w in self.weights]
        delta_biases = [np.zeros(b.shape) for b in self.biases]

        activations = [image.T]
        activation = image.T

        for w, b in zip(self.weights, self.biases):
            activation = sigmoid(np.dot(w, activation) + b)
            activations.append(activation)

        delta = self.compute_cost(label.T, activations)

        for i, (a_i, delta_j) in enumerate(zip(activations, delta)):
            delta_weights[i] = self.momentum * self.last_delta_w[i] + delta_weights[i] + np.dot(delta_j, a_i.T)
            delta_biases[i] = self.momentum * self.last_delta_b[i] + delta_biases[i] + delta_j.sum(axis=-1, keepdims=True)

        return delta_weights, delta_biases

    def predict(self, test_data):
        # test_data: [784]
        y = self.feed_forward(test_data[...,None]) # [784, 1] -> [10, 1]
        results = np.argmax(y, axis=0) # [0.1, 0.3, 0.2, 0.4, 0.9, 0.4, 0.3, 0.4, 0.1, 0.5] -> [4]

        return results.item() # 4

    def compute_cost(self, label, activation):
        delta = []
        w = self.weights[::-1]
        for i, a in enumerate(activation[-1:0:-1]): #從最後一個往回取到index=1

            if i == 0:
                delta.append((label - a) * a * (1.0 - a))
            else:
                delta.append(a * (1.0 - a) * (np.dot(w[i-1].T, delta[i-1])))

        return delta[::-1]


#### 初始化並訓練模型

In [7]:
model = neural_network([training_images.shape[-1], 1024, 256, 64, len(classes)])
model.train_model(training_images, 
                  training_labels, 
                  num_epoch=50, 
                  learning_rate=0.1, 
                  batch_size=8,
                  valid_images=valid_images,
                  valid_labels=valid_labels,
                  momentum=0.05)

  2%|▏         | 1/50 [00:20<16:42, 20.46s/it]

Epoch 1 / 50     Accuracy : 0.7356909239574816


  4%|▍         | 2/50 [00:40<16:07, 20.16s/it]

Epoch 2 / 50     Accuracy : 0.8771463614063778


  6%|▌         | 3/50 [01:00<15:46, 20.14s/it]

...

Epoch 48 / 50     Accuracy : 0.946443172526574


 98%|█████████▊| 49/50 [16:19<00:21, 21.33s/it]

Epoch 49 / 50     Accuracy : 0.947260834014718


100%|██████████| 50/50 [16:40<00:00, 20.01s/it]

Epoch 50 / 50     Accuracy : 0.947260834014718


#### 儲存模型權重
~~但我自己沒測試過能不能用就是了~~

In [8]:
# import pickle

# checkpoint = {}
# checkpoint['weights'] = model.weights
# checkpoint['biases'] = model.biases

# with open('checkpoint.pickle', 'wb') as f:
#   pickle.dump(checkpoint, f)

#### 預測助教給的test set，並將結果寫進txt檔

In [9]:
# import matplotlib.pyplot as plt
# import random
# from sklearn.utils import shuffle
  
# data_list = test_dataset('Testing_data1') # (image, file_name) * length of list

# with open('711183116.txt', 'w') as f:
#   for img, file_name in data_list:
#     result = model.predict(img)
#     print('{} {}\n'.format(file_name.split('.')[0], classes[result]))
#     f.write('{} {}\n'.format(file_name.split('.')[0], classes[result]))